In [2]:
import pandas as pd
import geopandas as gpd
import shapely as shp
import numpy as np
import xarray as xr 
import netCDF4
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as mticker
from scipy import interpolate
import scipy as sp
import scipy.signal as signal
import warnings
import cartopy.crs as ccrs
import cartopy
import glob
import logging, sys
import dask
import dask.dataframe as dd
from dask.distributed import Client
from dask import compute, delayed
from datetime import date
import datetime 
from multiprocessing.pool import Pool
from astropy.time import Time
from tqdm.notebook import tqdm
from pyproj import Proj, transform
import gps_time
import re
import json
import pickle
import statistics as stat
import sys 
import os
from PIL import Image
from scipy import ndimage
from scipy.interpolate import griddata
import json

warnings.filterwarnings('ignore')
logging.disable(sys.maxsize)
warnings.filterwarnings("ignore")

In [57]:
json_file = "/Users/rricker/GitLab/projects/chione/config.json"

with open('config.json', 'r') as json_file:
    config = json.load(json_file)

def generate_snow_netcdf(config):
    out_dir = "/Volumes/data2/projects/chione/snow_ex/"
    source_is2 = sorted(glob.glob(config["dir"]["is2_netcdf"] + "/" + '*.nc'))
    source_cs2 = sorted(glob.glob(config["dir"]["cs2_netcdf"] + "/" + '*.nc'))
    
    time_vec = [datetime.datetime.strptime(
        re.search("([0-9]{4}[0-9]{2}[0-9]{2})", d)[0], '%Y%m%d')+datetime.timedelta(hours=12) for d in source_is2]
    
    for i in range(0,len(source_is2)):
        data_cs2  = xr.open_dataset(source_cs2[i],decode_times=False)
        data_is2  = xr.open_dataset(source_is2[i],decode_times=False)
        if i == 0:
            tmp = data_cs2.copy()
            tmp.sea_ice_freeboard.attrs['long_name'] = 'Snow depth, difference between laser and sea ice freeboard'
            tmp = tmp.drop(["sea_ice_concentration","time_offset_acquisition","dist_acquisition","sea_ice_freeboard_std"])
            tmp = tmp.expand_dims({'time':len(source_cs2)}).assign_coords({'time': time_vec}).copy()
            tmp = tmp.rename(name_dict={'sea_ice_freeboard':'snow_depth'})
            tmp = tmp["snow_depth"].copy()
            
        tmp[i,:,:]= data_is2["snow_freeboard"] - data_cs2["sea_ice_freeboard"]
    
    t0 = re.search("([0-9]{4}[0-9]{2}[0-9]{2})", source_is2[0])[0]
    t1 = re.search("([0-9]{4}[0-9]{2}[0-9]{2})", source_is2[-1])[0]
    outfile = "SnowDepth-"+t0+"_"+t1+".nc"
    tmp.to_netcdf(path=out_dir + outfile)
        
    return tmp


In [58]:
tmp = generate_snow_netcdf(config)

In [ ]:
out_dir_plots = "/Volumes/data2/projects/chione/snow_ex/plots/"
data  = xr.open_dataset("/Volumes/data2/projects/chione/snow_ex/test.nc",decode_times=False)

snow_all = data["snow_depth"].data
snow_all_valid = snow_all[np.isfinite(snow_all)]

for t in data["time"]:
    snow_grid = data["snow_depth"][t,:,:].data
    snow_valid = snow_grid[np.isfinite(snow_grid)]
    
    fig, ax = plt.subplots(figsize=(5,5),facecolor='white')

    plt.hist(snow_all_valid,bins=96, range=(-0.2,1),density=True,alpha=0.5,color="grey",
             label='Annual mean: ' + "{:1.2f}".format(np.mean(snow_all_valid))+' m')

    plt.hist(snow_valid,bins=96, range=(-0.2,1),density=True,alpha=0.3,color="red",
             label='Daily mean: ' + "{:1.2f}".format(np.mean(snow_valid))+' m')

    ax.set_xlim([0,0.6])
    ax.set_ylim([0,10])
    
    plt.xlabel('Snow depth in meters',fontsize=12)
    plt.ylabel('Density',fontsize=12)
    plt.title('Arctic snow depth day '+"{:03d}".format(t.data),fontsize=12)

    ax.tick_params(axis = 'both', labelsize = 12)

    leg = ax.legend(loc="upper left",fontsize=12,markerscale=10,
                    facecolor='white',framealpha=0.4)

    plt.savefig(out_dir_plots+'hist_insitu'+"{:03d}".format(t.data)+'.png',dpi=150,bbox_inches='tight' )
    

In [3]:
out_dir_plots = "/Volumes/data2/projects/chione/is2_plots/"

def make_gif(png_dir):
    frames = []
    png_list = glob.glob(png_dir + '*' +"*.png")
    
    for frame in png_list:
        new_frame = Image.open(frame)
        frames.append(new_frame)
    
    out_name = ('test.gif')
    outfile = png_dir +  out_name

    frames[0].save(outfile, format='GIF',
                   append_images=frames[1:],
                   save_all=True,
                   duration=100, loop=0)
        
make_gif(out_dir_plots)



In [55]:
t0 = re.search("([0-9]{4}[0-9]{2}[0-9]{2})", source_is2[0])[0]
t1 = re.search("([0-9]{4}[0-9]{2}[0-9]{2})", source_is2[-1])[0]

In [ ]:
/Volumes/data2/projects/chione/is2_plots/ATL10-nh-20210219-v005-250_snow_freeboard.png